# Parameters 

In [1]:
fileName = '3PKk.hdf5'
# fileName = 'AllStates_intVec.hdf5'
nPi = 3
nPa = 1
nWPa = 1
input_shape = (4,8,8)
batch_size = 256
epochs = 300
dataDiv = 1
num_classes = 5

# Imports

In [2]:
# from __future__ import print_function
import keras
# from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
# import pandas as pd
from sklearn.model_selection import train_test_split
import sys



Using TensorFlow backend.


# Data Loader

In [3]:
# %load h5View.py
import sys
import h5py
import time
import os
import numpy as np
import signal

def db(t,v):
    if debug == True:
        print(t,v)
debug = False
debug = True

def Db(t,v):
    if Debug == True:
        print(t,v)
Debug = False
Debug = True

def loadDataSetFromFile(fileName, dataSetName):
    f = h5py.File(fileName, 'a')
    return f[dataSetName]

def loadDataSetFromHandle(f, dataSetName = None):
    ####################
    #
    #   Signal Handler
    #
    ####################
    def exit_gracefully(signum, frame):
        # restore the original signal handler as otherwise evil things will happen
        # in raw_input when CTRL+C is pressed, and our signal handler is not re-entrant
        signal.signal(signal.SIGINT, original_sigint)

        try:
            # if input("\nReally quit? (y/n)> ").lower().startswith('y'):
            #     print("Flushing data to disk")
            #     f.close()
            #     sys.exit(1)
            print("\nFlushing data to disk")
            f.close()

        except KeyboardInterrupt:
            print("\nOk ok, quitting")
            print("Flushing data to disk")
            f.close()
            sys.exit(1)

        # restore the exit gracefully handler here
        signal.signal(signal.SIGINT, exit_gracefully)

    # store the original SIGINT handler
    original_sigint = signal.getsignal(signal.SIGINT)
    signal.signal(signal.SIGINT, exit_gracefully)

    ####################
    #
    #   Main
    #
    ####################

    # f = h5py.File(fileName, 'a')
    # print("File loadded, f = " + fileName)
    
    if dataSetName != None:
    
        print("Datasets:")
        i = 0
        datasetNames = [ds for ds in f]
        for ds in datasetNames:
            print('Nr:' , i , '  Name:', ds.ljust(15), "Size:", f[ds].shape )
            i += 1
        # print(f[0])
        while True:
            selectDS = int(input("Select dataset: "))
            action = input(str("Do you want to load dataset: " + datasetNames[selectDS] + "? [y/n] "))
            if action == 'y' or action == 'Y':
                dataset = f[datasetNames[selectDS]]
                print("Dataset loaded")
                break
        return dataset
    else:
        return f[dataSetName]

def checkForTens(dataset):
    d = 1000
    l = len(dataset)
    p = int(l/d)

    for i in range(l):
        if dataset[i]==10:
            print('i: ', i, ' dataset[{}]'.format(i), dataset[i])
        if i%p==0:
            print(round(i/l,3))

def help():

    Db("Usage" +
       "\n    " +
       "Open file handle:".ljust(25) + "f = h5py.File(fileName,'a')" +
       "\n    " +
       "Close file handle:".ljust(25) + "f.close" +
       "\n    " +
       "Load dataset :".ljust(25) + "dataset = loadDataSetFromFile(fileHandle, dataSetName)" +
       "\n    " +
       "Load dataset interactively:".ljust(25) + "dataset = loadDataSetInteractive(f)" +
       "\n    " +
       "Display this help:". ljust(25) + "help()"
       ,'')

if __name__ == '__main__':
    ########################
    #
    #   Parameters
    #
    ########################

#     fileName = 'AllStates_intVec.hdf5'
#     dataSetName = "36KPvKPP"

    help()

#     db("\nOpening file:", fileName)
#     f = h5py.File(fileName, 'a')
#     dataset = 0
#     dataset = loadDataSetInteractive(f)
#     print("...to variable dataset")




Usage
    Open file handle:        f = h5py.File(fileName,'a')
    Close file handle:       f.close
    Load dataset :           dataset = loadDataSetFromFile(fileHandle, dataSetName)
    Load dataset interactively:dataset = loadDataSetInteractive(f)
    Display this help:       help() 


## Analysis
```
3PKk-Wdl-Buffered == 3PKk-Wdl-Seq

4PpKk-Wdl == 4PpKk-Wdl-Buffered != 4PpKk-Wdl-retry

5PPpKk-Wdl != 5PPpKk-Wdl-Buffered
    from 50.000 then buffered != 10, but wdl == 10
```
  
### WDL score count
#### 3PKk-WDL-Seq
[0, 0, 125024, 0, 124960]

# Functions for vector to full state conversion

In [4]:
def sq2hnit(sq):
    col = sq%8
    row = (sq - col)//8
    return col,row

# 0: pawns
# 1: kings
def vecSt2fullSt(vecSt, nPi, nPa, nWPa):
    fullSt = np.zeros((4,8,8), dtype = 'int8')
    for i in range(nPi - 2):
        sq = vecSt[i]
        col,row = sq2hnit(sq)
        if i < nWPa:
            fullSt[0][row][col] = 1
        else:
            fullSt[1][row][col] = -1
    col,row = sq2hnit(vecSt[-2])
    fullSt[2][row][col] = 1
    col,row = sq2hnit(vecSt[-1])
    fullSt[3][row][col] = -1
    return fullSt 

def vecSt2fullSt_8x8x2(vecSt, nPi, nPa, nWPa):
    fullSt = np.zeros((8,8,2), dtype = 'int8')
    for i in range(nPi - 2):
        sq = vecSt[i]
        col,row = sq2hnit(sq)
        if i < nWPa:
            fullSt[row][col][0] = 1
        else:
            fullSt[row][col][0] = -1
    col,row = sq2hnit(vecSt[-2])
    fullSt[row][col][1] = 1
    col,row = sq2hnit(vecSt[-1])
    fullSt[row][col][1] = -1
    return fullSt 
# nPi = 3
# nPa = 1
# nWPa = 1
# vecSt2fullSt(d3[0], nPi, nPa, nWPa)
# nPi = 5
# nPa = 3
# nWPa = 2
# vecSt2fullSt(d5[0], nPi, nPa, nWPa)
    

# Load data

In [5]:
# load datasets
f = h5py.File(fileName, 'a')
d3 = f['3PKk']
d3t = f['3PKk-Wdl-Seq']
# d5 = f['5PPpKk']
l = len(d3)



In [6]:
# count nr of each score instance
# wdlCounter placeholders: [-2, -1, 0, 1 ,2]

def wdlCountingMachine(ds):
    wdlCounter = [0,0,0,0,0]
    l = len(ds)
    i = 0
    intv = l//100
    for wdl in ds:
        i += 1
        if i%intv == 0:
            sys.stdout.write(str((i*100)//l) + " percentage")
            sys.stdout.write('\r')
            sys.stdout.flush()
        wdlCounter[wdl[0] + 2] += 1
    print(wdlCounter)
    return wdlCounter
# wdlCountingMachine(d3t)

In [7]:
X = np.array([vecSt2fullSt(vecSt,nPi, nPa, nWPa) for vecSt in d3[:l//dataDiv]])
y = d3t[:l//dataDiv]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [8]:
print(y_test.shape)

(82495, 1)


In [9]:
print('X_train shape:', X_train.shape)
print('y_train shape:', y_train.shape)
print('X_test shape:', X_test.shape)
print('y_test shape:', y_test.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


X_train shape: (167489, 4, 8, 8)
y_train shape: (167489, 1)
X_test shape: (82495, 4, 8, 8)
y_test shape: (82495, 1)
167489 train samples
82495 test samples


In [10]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Create Keras model

In [11]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),
                 padding='valid',
                 activation='relu',
                 data_format = "channels_first",
#                  kernel_initializer = 
                 input_shape=input_shape))
model.add(Conv2D(96, kernel_size=(3, 3),
                 padding='valid',
                 activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3),
                 padding='valid',
                 activation='relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 6, 6)          2368      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 4, 96)         5280      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 2, 256)        221440    
_________________________________________________________________
flatten_1 (Flatten)          (None, 30720)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 153605    
Total params: 382,693
Trainable params: 382,693
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


In [13]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 167489 samples, validate on 82495 samples
Epoch 1/300
167489/167489 [==============================] - 92s 549us/step - loss: 0.3408 - acc: 0.8330 - val_loss: 0.1340 - val_acc: 0.9447
Epoch 2/300
167489/167489 [==============================] - 91s 545us/step - loss: 0.1017 - acc: 0.9587 - val_loss: 0.0825 - val_acc: 0.9658
Epoch 3/300
167489/167489 [==============================] - 91s 545us/step - loss: 0.0649 - acc: 0.9737 - val_loss: 0.0552 - val_acc: 0.9779
Epoch 4/300
167489/167489 [==============================] - 97s 578us/step - loss: 0.0522 - acc: 0.9791 - val_loss: 0.0902 - val_acc: 0.9628
Epoch 5/300
167489/167489 [==============================] - 96s 575us/step - loss: 0.0438 - acc: 0.9823 - val_loss: 0.0534 - val_acc: 0.9790
Epoch 6/300
167489/167489 [==============================] - 97s 576us/step - loss: 0.0376 - acc: 0.9852 - val_loss: 0.0460 - val_acc: 0.9821
Epoch 7/300
167489/167489 [==============================] - 96s 575us/step - loss: 0.0335 - acc:

Epoch 58/300
167489/167489 [==============================] - 97s 580us/step - loss: 6.9862e-04 - acc: 0.9998 - val_loss: 0.0172 - val_acc: 0.9952
Epoch 59/300
167489/167489 [==============================] - 97s 579us/step - loss: 5.9295e-04 - acc: 0.9999 - val_loss: 0.0113 - val_acc: 0.9971
Epoch 60/300
167489/167489 [==============================] - 97s 579us/step - loss: 9.9897e-04 - acc: 0.9997 - val_loss: 0.0113 - val_acc: 0.9969
Epoch 61/300
167489/167489 [==============================] - 97s 579us/step - loss: 5.4656e-04 - acc: 0.9999 - val_loss: 0.0141 - val_acc: 0.9963
Epoch 62/300
167489/167489 [==============================] - 97s 577us/step - loss: 7.0158e-04 - acc: 0.9998 - val_loss: 0.0180 - val_acc: 0.9950
Epoch 63/300
167489/167489 [==============================] - 94s 563us/step - loss: 6.4156e-04 - acc: 0.9999 - val_loss: 0.0102 - val_acc: 0.9973
Epoch 64/300
167489/167489 [==============================] - 92s 552us/step - loss: 2.7725e-04 - acc: 1.0000 - val_lo

Epoch 114/300
167489/167489 [==============================] - 92s 550us/step - loss: 2.6460e-04 - acc: 0.9999 - val_loss: 0.0111 - val_acc: 0.9973
Epoch 115/300
167489/167489 [==============================] - 92s 551us/step - loss: 3.7726e-04 - acc: 0.9999 - val_loss: 0.0113 - val_acc: 0.9973
Epoch 116/300
167489/167489 [==============================] - 93s 552us/step - loss: 3.3528e-04 - acc: 0.9999 - val_loss: 0.0113 - val_acc: 0.9973
Epoch 117/300
167489/167489 [==============================] - 92s 552us/step - loss: 2.0756e-04 - acc: 0.9999 - val_loss: 0.0119 - val_acc: 0.9972
Epoch 118/300
167489/167489 [==============================] - 92s 552us/step - loss: 6.3013e-04 - acc: 0.9998 - val_loss: 0.0150 - val_acc: 0.9964
Epoch 119/300
167489/167489 [==============================] - 92s 551us/step - loss: 2.9502e-04 - acc: 0.9999 - val_loss: 0.0127 - val_acc: 0.9970
Epoch 120/300
167489/167489 [==============================] - 92s 552us/step - loss: 2.1290e-04 - acc: 1.0000 -

167489/167489 [==============================] - 92s 551us/step - loss: 2.4820e-04 - acc: 1.0000 - val_loss: 0.0141 - val_acc: 0.9970
Epoch 170/300
167489/167489 [==============================] - 92s 551us/step - loss: 2.0682e-04 - acc: 1.0000 - val_loss: 0.0125 - val_acc: 0.9973
Epoch 171/300
167489/167489 [==============================] - 92s 552us/step - loss: 1.5872e-04 - acc: 1.0000 - val_loss: 0.0125 - val_acc: 0.9973
Epoch 172/300
167489/167489 [==============================] - 92s 551us/step - loss: 2.1105e-04 - acc: 1.0000 - val_loss: 0.0129 - val_acc: 0.9972
Epoch 173/300
167489/167489 [==============================] - 92s 549us/step - loss: 1.0987e-04 - acc: 1.0000 - val_loss: 0.0126 - val_acc: 0.9974
Epoch 174/300
167489/167489 [==============================] - 92s 552us/step - loss: 2.4461e-04 - acc: 1.0000 - val_loss: 0.0126 - val_acc: 0.9973
Epoch 175/300
167489/167489 [==============================] - 93s 554us/step - loss: 2.9460e-04 - acc: 0.9999 - val_loss: 0.0

167489/167489 [==============================] - 92s 550us/step - loss: 1.4751e-04 - acc: 1.0000 - val_loss: 0.0128 - val_acc: 0.9972
Epoch 225/300
167489/167489 [==============================] - 92s 549us/step - loss: 2.6194e-04 - acc: 1.0000 - val_loss: 0.0127 - val_acc: 0.9971
Epoch 226/300
167489/167489 [==============================] - 93s 554us/step - loss: 1.5486e-04 - acc: 1.0000 - val_loss: 0.0159 - val_acc: 0.9966
Epoch 227/300
167489/167489 [==============================] - 93s 553us/step - loss: 2.5476e-04 - acc: 1.0000 - val_loss: 0.0126 - val_acc: 0.9973
Epoch 228/300
167489/167489 [==============================] - 94s 558us/step - loss: 1.9869e-04 - acc: 1.0000 - val_loss: 0.0126 - val_acc: 0.9973
Epoch 229/300
167489/167489 [==============================] - 92s 552us/step - loss: 2.2279e-04 - acc: 1.0000 - val_loss: 0.0122 - val_acc: 0.9973
Epoch 230/300
167489/167489 [==============================] - 92s 552us/step - loss: 2.1635e-04 - acc: 1.0000 - val_loss: 0.0

167489/167489 [==============================] - 93s 554us/step - loss: 2.0590e-04 - acc: 1.0000 - val_loss: 0.0131 - val_acc: 0.9972
Epoch 280/300
167489/167489 [==============================] - 93s 553us/step - loss: 1.8075e-04 - acc: 1.0000 - val_loss: 0.0141 - val_acc: 0.9971
Epoch 281/300
167489/167489 [==============================] - 93s 553us/step - loss: 2.3558e-04 - acc: 1.0000 - val_loss: 0.0131 - val_acc: 0.9973
Epoch 282/300
167489/167489 [==============================] - 93s 556us/step - loss: 1.4816e-04 - acc: 1.0000 - val_loss: 0.0131 - val_acc: 0.9973
Epoch 283/300
167489/167489 [==============================] - 92s 550us/step - loss: 1.7533e-04 - acc: 1.0000 - val_loss: 0.0134 - val_acc: 0.9971
Epoch 284/300
167489/167489 [==============================] - 93s 555us/step - loss: 2.0150e-04 - acc: 1.0000 - val_loss: 0.0132 - val_acc: 0.9972
Epoch 285/300
167489/167489 [==============================] - 93s 555us/step - loss: 1.9464e-04 - acc: 0.9999 - val_loss: 0.0

In [14]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02843609377655515
Test accuracy: 0.994751197042245
